# Whats wrong with MLE

In [1]:
using ForwardDiff
using GynC
using Plots
using StatPlots

In [302]:
using Parameters

@with_kw struct Model
  f = x->x.^2
  n = 100
  xs = linspace(1,2,n)
  ndata = 100
  σ = 0.1
  γ = 1
end

dataprior = TruncatedNormal(1.5, 0.1, 1, 2)
wtrue = normalize(pdf.(dataprior, m.xs), 1);

function generatedata(m, n; prior=dataprior)
  # smooth, true samples
  #dataxs = rand(prior, n)
  # discrete samples
  dataxs = m.xs[rand(Categorical(wtrue), n)]
  
  datays = m.f.(dataxs) + rand(Normal(0, m.σ), n)

  dataxs, datays
end

jeffreysprior(m) = normalize([ForwardDiff.derivative(m.f, x) for x in m.xs], 1)
  
likelihoodmat(m, data) = [pdf(Normal(y, m.σ), d) for d in data, y in m.f.(m.xs)]


logL(w, L)  = sum(log.(L * w)) 
dlogL(w, L) = sum(L ./ (L*w), 1) |> vec

dklj(w, j)  = sum(w[i] * log(w[i]/j[i]) for i in 1:length(w))
ddklj(w, j) = [log(w[i]/j[i]) + 1 for i in 1:length(w)]

function ebprior(m, datas, γ, c=GynC.OptConfig(n=MAXEVAL))
  
  data = datas[2]
  
  j = jeffreysprior(m)
  L = likelihoodmat(m, data)  
  
   obj(w) = (γ > 0 ? γ *  dklj(w,j) : 0) -  logL(w,L)
  dobj(w) = (γ > 0 ? γ * ddklj(w,j) : 0) - dlogL(w,L)
  
  opt = GynC.simplex_minimize(obj, dobj, ones(m.n), config=c)
  global opthist 
  opthist = opt[2]
  opt[1] 
end

ebprior (generic function with 2 methods)

In [307]:
m = Model()
datas = generatedata(m, 100000);
L = likelihoodmat(m, datas[2]);

In [309]:
#c = GynC.OptConfig(MAXEVAL=10)
c = GynC.OptConfig(FTOLREL=1e-8)
w = ebprior(m, datas, 0, c)

@show length(opthist)

@show logL(w, L)
@show logL(w, L) - logL(wtrue, L)
println()

@show norm(w-wtrue)
@show maximum(w)

plot(m.xs, w*100)
plot!(dataprior)

length(opthist) = 670
logL(w, L) = -26608.10243384813
logL(w, L) - logL(wtrue, L) = 14.198394950701186

norm(w - wtrue) = 0.07352141201672913
maximum(w) = 0.07310187112612106


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 2.00 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
<polyline clip-path="url(#clip4302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 22.3377,375.813 27.9738,375.813 33.6099,375.813 39.2461,375.813 44.8822,375.813 50.5183,375.813 56.1545,375.813 61.7906,375.813 67.4267,375.813 73.0629,375.813 
 78.699,375.813 84.3351,375.813 89.9713,375.813 95.6074,375.813 101.244,375.813 106.88,375.811 112.516,374.807 118.152,373.655 123.788,374.069 129.424,374.993 
 135.06,375.115 140.696,375.813 146.333,372.73 151.969,368.607 157.605,365.954 163.241,359.156 168.877,358.185 174.513,358.132 180.149,360.335 185.786,359.598 
 191.422,361.689 197.058,362.196 202.694,300.141 208.33,301.573 213.966,317.192 219.602,314.215 225.238,329.236 230.875,334.288 236.511,224.792 242.147,171.62 
 247.783,318.246 253.419,288.983 259.055,239.493 264.691,172.623 270.328,116.815 275.964,190.736 281.6,250.262 287.236,301.902 292.872,81.762 298.508,193.186 
 304.144,179.795 309.781,11.811 315.417,277.576 321.053,240.073 326.689,249.287 332.325,185.137 337.961,178.051 343.597,182.26 349.233,225.388 354.87,285.17 
 360.506,316.871 366.142,166.654 371.778,327.871 377.414,317.77 383.05,316.16 388.686,328.995 394.323,321.611 399.959,251.433 405.595,367.767 411.231,366.697 
 416.867,363.611 422.503,362.046 428.139,350.787 433.775,358.435 439.412,363.786 445.048,371.702 450.684,371.319 456.32,371.973 461.956,370.981 467.592,373.171 
 473.228,374.769 478.865,375.813 484.501,375.192 490.137,375.094 495.773,375.384 501.409,375.81 507.045,375.813 512.681,375.813 518.317,375.022 523.954,375.813 
 529.59,375.813 535.226,375.813 540.862,375.813 546.498,375.813 552.134,375.813 557.77,375.813 563.407,375.813 569.043,375.813 574.679,375.813 580.315,375.813 
 
 "/>
<polyline clip-path="url(#clip4302)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 27.7868,375.812 78.1716,375.747 108.451,375.308 134.051,373.593 147.956,371.269 161.861,367.074 168.27,364.244 174.68,360.699 181.089,356.325 187.499,351.016 
 194.371,344.175 201.242,336.053 208.114,326.599 214.985,315.814 222.947,301.749 230.908,286.228 238.87,269.639 246.831,252.514 253.906,237.377 260.982,222.859 
 268.057,209.515 275.132,197.891 281.336,189.512 287.541,183.136 293.745,178.99 299.949,177.225 303.332,177.293 306.714,178.088 310.096,179.603 313.478,181.82 
 320.243,188.258 327.007,197.128 333.811,208.132 340.616,220.782 347.421,234.593 354.225,249.073 369.272,281.169 384.319,310.094 391.477,321.955 398.635,332.397 
 405.793,341.385 412.952,348.958 420.395,355.43 427.837,360.615 435.28,364.681 442.723,367.803 455.254,371.392 467.785,373.493 498.382,375.425 527.144,375.758 
 573.224,375.812 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

In [306]:
#c = GynC.OptConfig(MAXEVAL=10)
c = GynC.OptConfig(FTOLREL=1e-7)
w = ebprior(m, datas, 0, c)

@show length(opthist)

@show logL(w, L)
@show logL(w, L) - logL(wtrue, L)
println()

@show norm(w-wtrue)
@show maximum(w)

plot(m.xs, w*100)
plot!(dataprior)

length(opthist) = 885
logL(w, L) = -2582.547450750406
logL(w, L) - logL(wtrue, L) = 10.54188323993776

norm(w - wtrue) = 0.11760495963986972
maximum(w) = 0.09084988364318734


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.00 
 
 
 1.25 
 
 
 1.50 
 
 
 1.75 
 
 
 2.00 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
<polyline clip-path="url(#clip3902)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 22.3377,375.813 27.9738,375.813 33.6099,375.813 39.2461,375.813 44.8822,375.813 50.5183,375.813 56.1545,375.813 61.7906,375.813 67.4267,375.813 73.0629,375.813 
 78.699,375.813 84.3351,375.813 89.9713,375.813 95.6074,375.813 101.244,375.813 106.88,372.466 112.516,375.813 118.152,375.813 123.788,375.813 129.424,375.813 
 135.06,375.813 140.696,375.813 146.333,375.813 151.969,375.813 157.605,364.553 163.241,342.304 168.877,362.015 174.513,375.813 180.149,375.813 185.786,375.813 
 191.422,375.813 197.058,367.148 202.694,289.26 208.33,230.188 213.966,317.776 219.602,370.304 225.238,375.813 230.875,375.813 236.511,329.071 242.147,180.834 
 247.783,138.425 253.419,243.644 259.055,256.316 264.691,275.669 270.328,375.813 275.964,353.878 281.6,214.807 287.236,168.602 292.872,11.811 298.508,141.704 
 304.144,180.862 309.781,375.81 315.417,298.899 321.053,265.37 326.689,252.929 332.325,113.822 337.961,188.462 343.597,210.327 349.233,375.648 354.87,314.203 
 360.506,290.687 366.142,234.17 371.778,297.306 377.414,333.986 383.05,338.391 388.686,336.634 394.323,333.188 399.959,362.103 405.595,360.279 411.231,325.682 
 416.867,314.165 422.503,374.396 428.139,375.813 433.775,375.813 439.412,375.812 445.048,370.303 450.684,358.254 456.32,375.813 461.956,375.813 467.592,375.813 
 473.228,368.689 478.865,375.447 484.501,375.813 490.137,375.813 495.773,375.813 501.409,375.813 507.045,375.813 512.681,375.813 518.317,375.813 523.954,375.813 
 529.59,375.813 535.226,375.813 540.862,375.813 546.498,375.813 552.134,375.813 557.77,375.813 563.407,375.813 569.043,375.813 574.679,375.813 580.315,375.813 
 
 "/>
<polyline clip-path="url(#clip3902)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 27.7868,375.812 78.1716,375.76 108.451,375.407 134.051,374.026 147.956,372.157 161.861,368.781 168.27,366.504 174.68,363.651 181.089,360.132 187.499,355.86 
 194.371,350.355 201.242,343.821 208.114,336.214 214.985,327.535 222.947,316.218 230.908,303.729 238.87,290.381 246.831,276.601 253.906,264.421 260.982,252.74 
 268.057,242.003 275.132,232.649 281.336,225.907 287.541,220.776 293.745,217.44 299.949,216.02 303.332,216.075 306.714,216.715 310.096,217.934 313.478,219.718 
 320.243,224.898 327.007,232.035 333.811,240.89 340.616,251.068 347.421,262.181 354.225,273.832 369.272,299.658 384.319,322.933 391.477,332.477 398.635,340.878 
 405.793,348.111 412.952,354.204 420.395,359.412 427.837,363.584 435.28,366.856 442.723,369.368 455.254,372.256 467.785,373.947 498.382,375.501 527.144,375.769 
 573.224,375.812 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

In [ ]:
function exp1(ndata::Int, optc)
  m = Model()
  datas = generatedata(m, ndata)
  w = ebprior(m, datas, 0, optc)
  global opthist
  w, opthist, norm(w-wtrue)/m.n, datas
end

c = GynC.OptConfig(FTOLREL=1e-5)
t = @elapsed(exps = map(n->exp1(n,c), 1000:5000:250000));

In [270]:
@show t;
plot(1000:5000:250000, map(r->length(r[2]), exps)) |> display

ws = getindex.(exps,[1])
plot(1000:5000:250000, map(w->norm(w - wtrue)/100, ws))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 60 
 
 
 70 
 
 
 80 
 
 
 90 
 
 
 100 
 
 
 110 
 
<polyline clip-path="url(#clip1902)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.5169,18.4292 46.6352,11.811 57.7536,157.412 68.8719,210.358 79.9902,190.503 91.1086,316.249 102.227,236.831 113.345,316.249 124.464,303.013 135.582,210.358 
 146.7,263.304 157.819,283.158 168.937,289.777 180.055,269.922 191.174,276.54 202.292,243.449 213.41,342.722 224.528,336.104 235.647,197.121 246.765,276.54 
 257.883,236.831 269.002,355.959 280.12,256.685 291.238,269.922 302.357,263.304 313.475,276.54 324.593,250.067 335.712,256.685 346.83,289.777 357.948,250.067 
 369.067,309.631 380.185,223.594 391.303,303.013 402.422,375.813 413.54,322.868 424.658,336.104 435.777,269.922 446.895,355.959 458.013,309.631 469.132,250.067 
 480.25,263.304 491.368,283.158 502.487,309.631 513.605,256.685 524.723,329.486 535.842,322.868 546.96,309.631 558.078,269.922 569.197,329.486 580.315,256.685 
 
 "/>
 
 
 
 
 y1

t = 923.965046953


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 200000 
 
 
 0.06 
 
 
 0.07 
 
 
 0.08 
 
 
 0.09 
 
 
 0.10 
 
<polyline clip-path="url(#clip2102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,11.811 49.8628,222.937 60.9139,375.813 71.965,302.136 83.0161,328.743 94.0672,290.41 105.118,302.586 116.169,209.05 127.22,278.496 138.272,336.025 
 149.323,294.109 160.374,178.965 171.425,303.012 182.476,302.816 193.527,235.355 204.578,228.113 215.629,260.322 226.68,290.971 237.731,218.089 248.782,199.681 
 259.833,249.392 270.885,299.891 281.936,223.908 292.987,185.341 304.038,271.179 315.089,150.257 326.14,194.468 337.191,228.375 348.242,231.82 359.293,244.067 
 370.344,225.546 381.395,222.714 392.446,179.641 403.498,252.977 414.549,230.317 425.6,247.226 436.651,172.035 447.702,255.756 458.753,204.252 469.804,181.29 
 480.855,170.653 491.906,166.851 502.957,244.355 514.008,176.519 525.06,183.822 536.111,249.833 547.162,199.97 558.213,236.995 569.264,196.276 580.315,102.18 
 
 "/>
 
 
 
 
 y1

In [271]:
plot(ws[end])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
<polyline clip-path="url(#clip2302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 22.3377,375.813 27.9738,375.812 33.6099,375.812 39.2461,375.812 44.8822,375.811 50.5183,375.81 56.1545,375.809 61.7906,375.807 67.4267,375.804 73.0629,375.799 
 78.699,375.787 84.3351,375.759 89.9713,375.676 95.6074,375.554 101.244,375.341 106.88,375.216 112.516,375.174 118.152,375.301 123.788,375.306 129.424,375.138 
 135.06,374.891 140.696,372.586 146.333,370.665 151.969,368.85 157.605,367.102 163.241,366.077 168.877,365.514 174.513,365.624 180.149,366.252 185.786,367.123 
 191.422,367.988 197.058,368.892 202.694,224.615 208.33,337.059 213.966,340.223 219.602,343.532 225.238,345.698 230.875,345.06 236.511,251.835 242.147,180.924 
 247.783,316.205 253.419,278.383 259.055,229.904 264.691,213.157 270.328,231.54 275.964,273.762 281.6,309.314 287.236,11.811 292.872,222.562 298.508,253.459 
 304.144,250.837 309.781,205.305 315.417,48.1757 321.053,296.295 326.689,264.947 332.325,229.913 337.961,208.57 343.597,225.743 349.233,261.824 354.87,295.612 
 360.506,328.439 366.142,140.087 371.778,343.355 377.414,343.21 383.05,343.921 388.686,341.86 394.323,319.712 399.959,261.169 405.595,368.61 411.231,368.759 
 416.867,366.302 422.503,360.638 428.139,360.014 433.775,361.3 439.412,365.215 445.048,370.058 450.684,372.176 456.32,371.739 461.956,373.422 467.592,374.81 
 473.228,374.516 478.865,374.555 484.501,374.995 490.137,375.308 495.773,375.502 501.409,375.668 507.045,375.707 512.681,375.732 518.317,375.746 523.954,375.752 
 529.59,375.753 535.226,375.759 540.862,375.777 546.498,375.8 552.134,375.809 557.77,375.811 563.407,375.812 569.043,375.813 574.679,375.813 580.315,375.813 
 
 "/>
 
 
 
 
 y1

In [247]:
plot(map(w->cumsum(abs(w-wtrue)),ws[1:5:end]))

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,1}) at ./deprecated.jl:57
 [3] _collect(::Array{Array{Float64,1},1}, ::Base.Generator{Array{Array{Float64,1},1},##319#320}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:488
 [4] map(::Function, ::Array{Array{Float64,1},1}) at ./abstractarray.jl:1868
 [5] include_string(::String, ::String) at ./loading.jl:522
 [6] include_string(::Module, ::String, ::String) at /home/numerik/bzfsikor/.julia/v0.6/Compat/src/Compat.jl:174
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/numerik/bzfsikor/.julia/v0.6/IJulia/src/execute_request.jl:154
 [8] (::Compat.#inner#16{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/numerik/bzfsikor/.julia/v0.6/Compat/src/Compat.jl:496
 [9] eventloop(::ZMQ.Socket) at /home/numerik/bzfsikor/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[247], in expression starting on line 1


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
<polyline clip-path="url(#clip8602)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 28.9273,375.813 34.4969,375.813 40.0664,375.813 45.636,375.813 51.2056,375.812 56.7752,375.811 62.3447,375.81 67.9143,375.808 73.4839,375.805 79.0534,375.801 
 84.623,375.794 90.1926,375.644 95.7622,375.352 101.332,375.333 106.901,375.301 112.471,375.256 118.04,375.193 123.61,375.104 129.18,374.98 134.749,374.811 
 140.319,374.581 145.888,374.411 151.458,373.112 157.027,372.947 162.597,372.245 168.167,371.335 173.736,370.17 179.306,368.697 184.875,366.854 190.445,364.57 
 196.014,361.77 201.584,358.395 207.154,355.602 212.723,332.528 218.293,330.772 223.862,327.951 229.432,320.324 235.001,311.597 240.571,301.733 246.141,293.698 
 251.71,287.076 257.28,281.444 262.849,232.353 268.419,222.3 273.988,219.257 279.558,202.213 285.128,183.976 290.697,175.134 296.267,170.894 301.836,146.796 
 307.406,124.688 312.975,119.819 318.545,118.783 324.115,109.647 329.684,98.6188 335.254,88.8165 340.823,81.109 346.393,78.7693 351.962,65.5557 357.532,59.5044 
 363.102,54.0174 368.671,52.4576 374.241,52.1049 379.81,47.1503 385.38,43.2521 390.949,41.1504 396.519,39.2996 402.089,25.8719 407.658,25.117 413.228,23.4232 
 418.797,22.1852 424.367,20.8531 429.937,19.6242 435.506,18.4598 441.076,17.5484 446.645,17.309 452.215,13.299 457.784,12.9382 463.354,12.6291 468.924,12.3992 
 474.493,12.2298 480.063,12.1064 485.632,12.0174 491.202,11.9538 496.771,11.9088 502.341,11.8774 507.911,11.8556 513.48,11.8407 519.05,11.8305 524.619,11.8237 
 530.189,11.8192 535.758,11.8162 541.328,11.8143 546.898,11.813 552.467,11.8122 558.037,11.8117 563.606,11.8114 569.176,11.8112 574.745,11.8111 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip8602)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 28.9273,375.813 34.4969,375.813 40.0664,375.813 45.636,375.813 51.2056,375.813 56.7752,375.812 62.3447,375.811 67.9143,375.81 73.4839,375.808 79.0534,375.804 
 84.623,375.798 90.1926,375.789 95.7622,375.776 101.332,375.756 106.901,375.73 112.471,375.698 118.04,375.657 123.61,375.608 129.18,375.548 134.749,375.45 
 140.319,375.283 145.888,375.031 151.458,374.694 157.027,374.462 162.597,374.184 168.167,373.879 173.736,373.858 179.306,373.682 184.875,373.13 190.445,372.88 
 196.014,371.223 201.584,368.707 207.154,365.473 212.723,359.028 218.293,358.99 223.862,356.146 229.432,351.976 235.001,346.347 240.571,339.037 246.141,326.451 
 251.71,323.783 257.28,317.866 262.849,311.684 268.419,304.406 273.988,295.824 279.558,266.819 285.128,254.664 290.697,252.425 296.267,245.415 301.836,238.666 
 307.406,233.402 312.975,229.077 318.545,227.417 324.115,199.048 329.684,190.395 335.254,185.234 340.823,183.88 346.393,183.615 351.962,182.284 357.532,178.961 
 363.102,173.986 368.671,155.147 374.241,150.136 379.81,145.943 385.38,143.186 390.949,142.685 396.519,136.358 402.089,133.12 407.658,130.641 413.228,128.984 
 418.797,128.341 424.367,127.756 429.937,127.645 435.506,127.38 441.076,127.171 446.645,126.686 452.215,126.499 457.784,126.442 463.354,126.294 468.924,125.912 
 474.493,125.832 480.063,125.743 485.632,125.658 491.202,125.599 496.771,125.559 502.341,125.545 507.911,125.537 513.48,125.525 519.05,125.524 524.619,125.519 
 530.189,125.516 535.758,125.513 541.328,125.512 546.898,125.511 552.467,125.51 558.037,125.51 563.606,125.51 569.176,125.51 574.745,125.51 580.315,125.51 
 
 "/>
<polyline clip-path="url(#clip8602)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 28.9273,375.813 34.4969,375.813 40.0664,375.813 45.636,375.813 51.2056,375.813 56.7752,375.812 62.3447,375.811 67.9143,375.809 73.4839,375.806 79.0534,375.801 
 84.623,375.795 90.1926,375.785 95.7622,375.77 101.332,37